## need: df with osrm and weather

#### train process
1. import csv to df
1. process df to be modelable
  1. add variables
    * dow (cat - 7 vars)
    * hr (cat - 24 vars)
        * pair dow with hr?
    * slow_day (cat - 0, 1)
    * fast_day (cat - 0, 1)
    * L2
        * log L2?
    * google_dist (needs id?)
    * weather (needs datetime?)
1. model
1. use cooksd + fox criterion to identify outliers
1. filter outliers and make new df

## * 아래 2칸 실행

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

from datetime import datetime

In [2]:
# 경로 변수 (고정)


trainfile = '../_data/train_03_997.csv'
old_trainfile = '../_data/train.csv'
kaggle_trainfile = '../_data/train_k.csv'

testfile = '../_data/test_clean.csv'
old_testfile = '../_data/test.csv'

train_171016 = '../_data/171016_dow_hr_pas_L2_dur_only.csv'
train_osrm = '../_data/train_with_osrm.csv'
train_osrm_weather = '../_data/train_with_osrm_and_weather.csv'
train_osrm_weather_hourly = '../_data/train_with_osrm_and_weather_hourly.csv'
train_dropped_nan_fixed = '../_data/train__dropped__no_disaster_nan_fixed.csv'
train_added_airps = '../_data/train__dropped__lnglat_saved__airports__no_disaster.csv'
train_added_airps_cleaned = '../_data/train__dropped__cleaned__airports__no_disaster.csv'
train_added_airps_disas_cleaned = '../_data/train__dropped__cleaned__eval__airports__disaster.csv'
train_added_weekhr = '../_data/train__dropped__cleaned__eval__a_d_weekhour.csv'
train_added_manh = '../_data/train__dropped__air_bliz_weekhr_manh.csv'
train_all_vars_rough = '../_data/train__all_vars__rough.csv'

train_701778_all_vars = '../_data/train__701778__all_vars.csv'
train_700806_all_vars_cooks_dropped = '../_data/train_700806_dropped_cooks.csv'
train_700789_all_vars_cooks_dropped_circle_filter = '../_data/train_700789_dropped_cooks__circle_filter.csv'
train_700789_all_vars_cdcf_kmeans = '../_data/train_kmeans_30_added.csv'
train_700419_all_vars_cdcf150kmh_km60 = '../_data/train_kmeans_60_added_cooksd_circle_150kmh.csv'

train_700419_all_vars_4d_clust120 = '../_data/train_kmeans_4D_120.csv'
train_700419_all_vars_4d_clust120_speed = '../_data/train_kmeans_4D_120_clustspeed.csv'
train_700419_all_vars_4d_clust120_speed_glob = '../_data/train_kmeans_4D_120_clustspeed_globalized.csv'

train_700419 = '../_data/train_700419_recent.csv'
train_697355 = '../_data/train_697355_rsq_0722.csv'
train_697349 = '../_data/train_697349_rsq_0723.csv'
train_697284 = '../_data/train_697284_rsq_0725.csv'
train_696688 = '../_data/train_696688_rsq_0731.csv'

test_recent = '../_data/test_backup_raw_osrm_dailyw_hourlyw.csv'

trains = [train_700419, train_697355, train_697349, train_697284, train_696688]

train_latest = trains[0]

## 원본 `test.csv` (전처리 이전)

In [4]:
df_test = pd.read_csv(test_recent,
                        infer_datetime_format=True,
                        parse_dates=['date'])
df_test.head(10)

,date,is_hol,m_d,hr,pas_cnt,p_lng,p_lat,d_lng,d_lat,total_dist,...,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,2016-06-30,False,630,23,5,-73.998291,40.722618,-73.971779,40.762470,5280.3,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,2016-06-30,False,630,23,2,-73.955109,40.689564,-73.978203,40.685802,2573.5,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,2016-06-30,False,630,23,1,-73.978416,40.791576,-73.973160,40.675968,15048.8,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,2016-06-30,False,630,23,2,-73.873093,40.774097,-73.926704,40.856739,16818.3,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,2016-06-30,False,630,23,1,-73.979416,40.755211,-74.003548,40.738224,3536.7,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
5,2016-06-30,False,630,23,1,-73.957794,40.776001,-73.982277,40.762093,3450.8,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
6,2016-06-30,False,630,23,1,-73.972694,40.764622,-73.972076,40.748665,2413.1,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
7,2016-06-30,False,630,23,2,-73.991402,40.750122,-73.989288,40.773579,4322.9,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
8,2016-06-30,False,630,23,1,-73.982750,40.745289,-73.969002,40.757950,2291.0,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
9,2016-06-30,False,630,23,2,-73.991631,40.727077,-74.003799,40.723572,1557.4,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


In [5]:
df_test.shape

(346797, 22)

In [6]:
def between(a, num, b):
    return (a < num) & (num < b)

In [7]:
between(3, 5, 5)

False

In [8]:
df_copy = df_test.copy()

In [9]:
didnt_move = pd.Series(np.zeros(df_copy.shape[0]), name='didnt_move')

In [10]:
df_copy.insert(0, 'didnt_move', didnt_move)
df_copy.head()

,didnt_move,date,is_hol,m_d,hr,pas_cnt,p_lng,p_lat,d_lng,d_lat,...,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,0.0,2016-06-30,False,630,23,5,-73.998291,40.722618,-73.971779,40.762470,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,0.0,2016-06-30,False,630,23,2,-73.955109,40.689564,-73.978203,40.685802,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,0.0,2016-06-30,False,630,23,1,-73.978416,40.791576,-73.973160,40.675968,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,0.0,2016-06-30,False,630,23,2,-73.873093,40.774097,-73.926704,40.856739,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,0.0,2016-06-30,False,630,23,1,-73.979416,40.755211,-74.003548,40.738224,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


In [11]:
df_copy.loc[between(-0.1, df_copy['total_dist'], 0.1), 'didnt_move'] = 1

In [12]:
df_copy['didnt_move'].value_counts()

0.0    345295
1.0      1502
Name: didnt_move, dtype: int64

In [13]:
df_copy.head()

,didnt_move,date,is_hol,m_d,hr,pas_cnt,p_lng,p_lat,d_lng,d_lat,...,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,0.0,2016-06-30,False,630,23,5,-73.998291,40.722618,-73.971779,40.762470,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,0.0,2016-06-30,False,630,23,2,-73.955109,40.689564,-73.978203,40.685802,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,0.0,2016-06-30,False,630,23,1,-73.978416,40.791576,-73.973160,40.675968,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,0.0,2016-06-30,False,630,23,2,-73.873093,40.774097,-73.926704,40.856739,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,0.0,2016-06-30,False,630,23,1,-73.979416,40.755211,-74.003548,40.738224,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


### to add: dow, m_d, hr, L2, holiday, disaster
## * dow

In [14]:
df_pdt = pd.read_csv('../_data/test.csv')

In [15]:
s_dow = df_pdt['pickup_datetime'].apply(pd.to_datetime)

In [16]:
s_time = s_dow.copy()

In [17]:
s_dow = s_time.dt.weekday_name

In [18]:
s_dow.head()

0    Thursday
1    Thursday
2    Thursday
3    Thursday
4    Thursday
Name: pickup_datetime, dtype: object

In [19]:
df_copy.insert(4, 'dow', s_dow)

In [21]:
pd.options.display.max_columns = 50

In [22]:
df_copy.head()

,didnt_move,date,is_hol,m_d,dow,hr,pas_cnt,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,0.0,2016-06-30,False,630,Thursday,23,5,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,0.0,2016-06-30,False,630,Thursday,23,2,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,0.0,2016-06-30,False,630,Thursday,23,1,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,0.0,2016-06-30,False,630,Thursday,23,2,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,0.0,2016-06-30,False,630,Thursday,23,1,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


### AIRPS

In [23]:
#JFK

#lng : 왼쪽, 오른쪽
#lat : 위, 아래

JFK_whole = {'lng' :  [-73.79139709, -73.77679442], 'lat' : [40.649094, 40.64365]}

#밀도 높은 부분들 중에서 왼쪽 위 부터 시계 방향으로

JFK_area1 = {'lng' : [-73.79139709, -73.78795624], 'lat' : [40.647731, 40.646172]}
JFK_area2 = {'lng' : [-73.785692, -73.781648], 'lat' : [40.649094, 40.647995]}
JFK_area3 = {'lng' : [-73.779073, -73.77679442], 'lat' : [40.646994, 40.643787]}
JFK_area4 = {'lng' : [-73.785295, -73.779835], 'lat' : [40.64698, 40.64294]}
JFK_area5 = {'lng' : [-73.791168, -73.787549], 'lat' : [40.644526, 40.4064365]}

JFK_areas = list()

for i in range(1, 6):
    eval('JFK_area{}[\'lat\'].sort()'.format(i))
    eval('JFK_areas.append(JFK_area{})'.format(i))


print(JFK_areas)
    

[{'lng': [-73.79139709, -73.78795624], 'lat': [40.646172, 40.647731]}, {'lng': [-73.785692, -73.781648], 'lat': [40.647995, 40.649094]}, {'lng': [-73.779073, -73.77679442], 'lat': [40.643787, 40.646994]}, {'lng': [-73.785295, -73.779835], 'lat': [40.64294, 40.64698]}, {'lng': [-73.791168, -73.787549], 'lat': [40.4064365, 40.644526]}]


In [24]:
# LaGuardia Airport

#lng : 왼쪽, 오른쪽
#lat : 위, 아래

LGA_whole = {'lng' :  [-73.8862, -73.8617], 'lat' : [40.774811, 40.767603]}

#밀도 높은 부분들 중에서 왼쪽 부터 오른쪽 방향으로

LGA_area1 = {'lng' : [-73.8862, -73.884859], 'lat' : [40.773526, 40.772366]}
LGA_area2 = {'lng' : [-73.876338, -73.870159], 'lat' : [40.774811, 40.773353]}
LGA_area3 = {'lng' : [ -73.866019, -73.8617], 'lat' : [40.771225, 40.767603]}

LGA_areas = list()

for i in range(1, 4):
    eval('LGA_area{}[\'lat\'].sort()'.format(i))
    eval('LGA_areas.append(LGA_area{})'.format(i))
    
print(LGA_areas)

[{'lng': [-73.8862, -73.884859], 'lat': [40.772366, 40.773526]}, {'lng': [-73.876338, -73.870159], 'lat': [40.773353, 40.774811]}, {'lng': [-73.866019, -73.8617], 'lat': [40.767603, 40.771225]}]


In [25]:
def within_range(lng, lat, lng_range, lat_range):
    return (lng_range[0] < lng < lng_range[1])\
           and (lat_range[0] < lat < lat_range[1])

def in_LGA(lng, lat):
    return sum(list(map(lambda x: within_range(lng, lat, x['lng'], x['lat']), LGA_areas))) == 1

def in_JFK(lng, lat):
    return sum(list(map(lambda x: within_range(lng, lat, x['lng'], x['lat']), JFK_areas))) == 1

def in_airport(lng, lat):
    return in_LGA(lng, lat) or in_JFK(lng, lat)

in_airport(-73.785, 40.647996)

True

In [26]:
df_copy.head()

,didnt_move,date,is_hol,m_d,dow,hr,pas_cnt,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,0.0,2016-06-30,False,630,Thursday,23,5,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,0.0,2016-06-30,False,630,Thursday,23,2,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,0.0,2016-06-30,False,630,Thursday,23,1,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,0.0,2016-06-30,False,630,Thursday,23,2,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,0.0,2016-06-30,False,630,Thursday,23,1,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


In [47]:
# p_airp, d_airp
df_train.insert(6, 'p_airp', np.vectorize(in_airport)(df_train['p_lng'], df_train['p_lat']))


In [54]:
del df_train['p_airp']

In [56]:
df_copy.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,p_lng,p_lat,d_lng,d_lat,dur,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,False,430,Saturday,23,0.069424,1,-73.987793,40.724792,-73.975616,40.656445,1454,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,-73.957596,40.717770,-73.951424,40.775230,1409,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,-74.000954,40.742031,-73.947708,40.782200,1081,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,-73.985733,40.738258,-73.993179,40.754890,800,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,-74.006615,40.740650,-73.985619,40.723362,1151,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


In [27]:
airports = ['LGA', 'JFK']

for airp in airports:
    df_copy.insert(6, 'p_{}'.format(airp),\
                    eval('np.vectorize(in_{})(df_copy[\'p_lng\'], df_copy[\'p_lat\'])'\
                         .format(airp)))



In [64]:
for airp in airports:
    df_copy.insert(6, 'p_{}'.format(airp),\
                    eval('np.vectorize(in_{})(df_copy[\'p_lng\'], df_copy[\'p_lat\'])'\
                         .format(airp)))

16771

In [28]:
for airp in airports:
    df_copy.insert(8, 'd_{}'.format(airp),\
                    eval('np.vectorize(in_{})(df_copy[\'d_lng\'], df_copy[\'d_lat\'])'\
                         .format(airp)))

In [29]:
df_copy['d_LGA'].sum()

4132

In [30]:
df_copy.head()

,didnt_move,date,is_hol,m_d,dow,hr,p_JFK,p_LGA,d_JFK,d_LGA,pas_cnt,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,0.0,2016-06-30,False,630,Thursday,23,False,False,False,False,5,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,0.0,2016-06-30,False,630,Thursday,23,False,False,False,False,2,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,0.0,2016-06-30,False,630,Thursday,23,False,False,False,False,1,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,0.0,2016-06-30,False,630,Thursday,23,False,True,False,False,2,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,0.0,2016-06-30,False,630,Thursday,23,False,False,False,False,1,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


In [31]:
df_copy.isnull().sum()

didnt_move       0
date             0
is_hol           0
m_d              0
dow              0
hr               0
p_JFK            0
p_LGA            0
d_JFK            0
d_LGA            0
pas_cnt          0
p_lng            0
p_lat            0
d_lng            0
d_lat            0
total_dist       0
total_time       0
day_max_temp     0
day_min_temp     0
day_rainfall     0
day_snowfall     0
day_snowdepth    0
hr_temp          0
hr_humid         0
hr_dewpt         0
hr_windspeed     0
hr_gustspeed     0
hr_precip        0
dtype: int64

In [32]:
df_copy.head()

,didnt_move,date,is_hol,m_d,dow,hr,p_JFK,p_LGA,d_JFK,d_LGA,pas_cnt,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,0.0,2016-06-30,False,630,Thursday,23,False,False,False,False,5,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,0.0,2016-06-30,False,630,Thursday,23,False,False,False,False,2,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,0.0,2016-06-30,False,630,Thursday,23,False,False,False,False,1,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,0.0,2016-06-30,False,630,Thursday,23,False,True,False,False,2,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,0.0,2016-06-30,False,630,Thursday,23,False,False,False,False,1,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


In [33]:
df_copy.to_csv('../_data/test_backup_clean_todo_clust.csv', index=False)

In [33]:
df_test.to_csv('../_data/test_backup_raw_osrm.csv', index=False)

In [37]:
df_w = pd.read_csv('../_data/_weather.csv',
                   infer_datetime_format=True,
                   parse_dates=['date'])

df_w.head()                   

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,2016-01-01,42,34,38.0,0.0,0.0,0.0
1,2016-01-02,40,32,36.0,0.0,0.0,0.0
2,2016-01-03,45,35,40.0,0.0,0.0,0.0
3,2016-01-04,36,14,25.0,0.0,0.0,0.0
4,2016-01-05,29,11,20.0,0.0,0.0,0.0


In [38]:
df_copy = df_test.copy()

In [41]:
from pandas import Timestamp

In [42]:
df_copy.insert(0, 'date', df_test['pickup_datetime']\
                          .apply(lambda x: Timestamp(x.year, x.month, x.day, 0, 0, 0)))

In [43]:
df_copy.head()

,date,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,total_distance,total_travel_time
0,2016-06-30,id2332349,2,2016-06-30 23:59:00,5,-73.998291,40.722618,-73.971779,40.762470,N,5280.3,623.3
1,2016-06-30,id2013516,1,2016-06-30 23:59:00,2,-73.955109,40.689564,-73.978203,40.685802,N,2573.5,219.3
2,2016-06-30,id3719493,2,2016-06-30 23:59:00,1,-73.978416,40.791576,-73.973160,40.675968,N,15048.8,1300.0
3,2016-06-30,id0376262,1,2016-06-30 23:59:00,2,-73.873093,40.774097,-73.926704,40.856739,N,16818.3,1029.1
4,2016-06-30,id2105228,1,2016-06-30 23:58:00,1,-73.979416,40.755211,-74.003548,40.738224,N,3536.7,434.5


In [44]:
df_copy = df_copy.merge(df_w, how='left', on='date')

In [45]:
df_copy.shape

(346797, 18)

In [46]:
df_copy.to_csv('../_data/test_backup_raw_osrm_dailyw.csv')

In [47]:
df_hourly = pd.read_csv('../_data/_metars.csv')

df_hourly.head()

,Time,Temp.,Windchill,Heat Index,Humidity,Pressure,Dew Point,Visibility,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,2015-12-31 02:00:00,7.8,7.1,NaN,0.89,1017.0,6.1,8.0,NNE,5.6,0.0,0.8,None,Overcast
1,2015-12-31 03:00:00,7.2,5.9,NaN,0.90,1016.5,5.6,12.9,Variable,7.4,0.0,0.3,None,Overcast
2,2015-12-31 04:00:00,7.2,NaN,NaN,0.90,1016.7,5.6,12.9,Calm,0.0,0.0,0.0,None,Overcast
3,2015-12-31 05:00:00,7.2,5.9,NaN,0.86,1015.9,5.0,14.5,NW,7.4,0.0,0.0,None,Overcast
4,2015-12-31 06:00:00,7.2,6.4,NaN,0.90,1016.2,5.6,11.3,West,5.6,0.0,0.0,None,Overcast


In [48]:
df_hourly.shape

(8787, 14)

In [49]:
for col in df_hourly.columns:
    print(col, df_hourly[col].isnull().sum())

Time 0
Temp. 0
Windchill 6492
Heat Index 7972
Humidity 0
Pressure 231
Dew Point 0
Visibility 237
Wind Dir 0
Wind Speed 0
Gust Speed 0
Precip 0
Events 0
Conditions 0


In [50]:
for col in ['Windchill', 'Heat Index', 'Pressure', 'Visibility']:
    del df_hourly[col]

In [51]:
df_hourly.head()

,Time,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,2015-12-31 02:00:00,7.8,0.89,6.1,NNE,5.6,0.0,0.8,None,Overcast
1,2015-12-31 03:00:00,7.2,0.90,5.6,Variable,7.4,0.0,0.3,None,Overcast
2,2015-12-31 04:00:00,7.2,0.90,5.6,Calm,0.0,0.0,0.0,None,Overcast
3,2015-12-31 05:00:00,7.2,0.86,5.0,NW,7.4,0.0,0.0,None,Overcast
4,2015-12-31 06:00:00,7.2,0.90,5.6,West,5.6,0.0,0.0,None,Overcast


In [52]:
for col in df_hourly.columns:
    print(col, df_hourly[col].isnull().sum())

Time 0
Temp. 0
Humidity 0
Dew Point 0
Wind Dir 0
Wind Speed 0
Gust Speed 0
Precip 0
Events 0
Conditions 0


In [53]:
df_hourly['Time'] = df_hourly['Time'].apply(pd.to_datetime)
df_hourly.head()

,Time,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,2015-12-31 02:00:00,7.8,0.89,6.1,NNE,5.6,0.0,0.8,None,Overcast
1,2015-12-31 03:00:00,7.2,0.90,5.6,Variable,7.4,0.0,0.3,None,Overcast
2,2015-12-31 04:00:00,7.2,0.90,5.6,Calm,0.0,0.0,0.0,None,Overcast
3,2015-12-31 05:00:00,7.2,0.86,5.0,NW,7.4,0.0,0.0,None,Overcast
4,2015-12-31 06:00:00,7.2,0.90,5.6,West,5.6,0.0,0.0,None,Overcast


In [69]:
df_copy = df_copy.merge(df_hourly, how='left', on='Time')


In [70]:
df_copy.head()

,date,id,Time,vendor_id,is_hol,m_d,pickup_datetime,hr,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,total_distance,total_travel_time,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,2016-06-30,id2332349,2016-06-30 23:00:00,2,False,630,2016-06-30 23:59:00,23,5,-73.998291,40.722618,-73.971779,40.762470,N,5280.3,623.3,85,67,76.0,0.0,0.0,0.0,24.4,0.6,16.1,Calm,0.0,0.0,0.0,None,Clear
1,2016-06-30,id2013516,2016-06-30 23:00:00,1,False,630,2016-06-30 23:59:00,23,2,-73.955109,40.689564,-73.978203,40.685802,N,2573.5,219.3,85,67,76.0,0.0,0.0,0.0,24.4,0.6,16.1,Calm,0.0,0.0,0.0,None,Clear
2,2016-06-30,id3719493,2016-06-30 23:00:00,2,False,630,2016-06-30 23:59:00,23,1,-73.978416,40.791576,-73.973160,40.675968,N,15048.8,1300.0,85,67,76.0,0.0,0.0,0.0,24.4,0.6,16.1,Calm,0.0,0.0,0.0,None,Clear
3,2016-06-30,id0376262,2016-06-30 23:00:00,1,False,630,2016-06-30 23:59:00,23,2,-73.873093,40.774097,-73.926704,40.856739,N,16818.3,1029.1,85,67,76.0,0.0,0.0,0.0,24.4,0.6,16.1,Calm,0.0,0.0,0.0,None,Clear
4,2016-06-30,id2105228,2016-06-30 23:00:00,1,False,630,2016-06-30 23:58:00,23,1,-73.979416,40.755211,-74.003548,40.738224,N,3536.7,434.5,85,67,76.0,0.0,0.0,0.0,24.4,0.6,16.1,Calm,0.0,0.0,0.0,None,Clear


In [19]:
pd.Timestamp(2015, 12, 31, 2)

Timestamp('2015-12-31 02:00:00')

In [22]:
del df_train['date']

In [24]:
df_copy = df_train.copy()

In [25]:
df_copy.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,id3495688,1,2016-04-30 23:59:00,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0
1,id3051282,1,2016-04-30 23:59:00,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,...,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0
2,id0082851,1,2016-04-30 23:59:00,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,...,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0
3,id0515725,2,2016-04-30 23:58:00,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,...,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0
4,id1872374,2,2016-04-30 23:58:00,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0


In [29]:
df_copy['pickup_datetime'] = df_copy['pickup_datetime'].apply(pd.to_datetime)

In [30]:
df_train = df_copy.copy()

In [31]:
df_copy.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,id3495688,1,2016-04-30 23:59:00,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0
1,id3051282,1,2016-04-30 23:59:00,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,...,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0
2,id0082851,1,2016-04-30 23:59:00,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,...,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0
3,id0515725,2,2016-04-30 23:58:00,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,...,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0
4,id1872374,2,2016-04-30 23:58:00,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0


In [54]:
def make_hourly(timestamp):
    y = timestamp.year
    m = timestamp.month
    d = timestamp.day
    h = timestamp.hour
    return pd.Timestamp(y, m, d, h, 0, 0)

In [55]:
df_copy.insert(2, 'Time', df_copy['pickup_datetime'].apply(make_hourly))

In [44]:
del df_copy['pickup_hour']

In [56]:
df_copy.head()

,date,id,Time,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,total_distance,total_travel_time,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,2016-06-30,id2332349,2016-06-30 23:00:00,2,2016-06-30 23:59:00,5,-73.998291,40.722618,-73.971779,40.762470,N,5280.3,623.3,85,67,76.0,0.0,0.0,0.0
1,2016-06-30,id2013516,2016-06-30 23:00:00,1,2016-06-30 23:59:00,2,-73.955109,40.689564,-73.978203,40.685802,N,2573.5,219.3,85,67,76.0,0.0,0.0,0.0
2,2016-06-30,id3719493,2016-06-30 23:00:00,2,2016-06-30 23:59:00,1,-73.978416,40.791576,-73.973160,40.675968,N,15048.8,1300.0,85,67,76.0,0.0,0.0,0.0
3,2016-06-30,id0376262,2016-06-30 23:00:00,1,2016-06-30 23:59:00,2,-73.873093,40.774097,-73.926704,40.856739,N,16818.3,1029.1,85,67,76.0,0.0,0.0,0.0
4,2016-06-30,id2105228,2016-06-30 23:00:00,1,2016-06-30 23:58:00,1,-73.979416,40.755211,-74.003548,40.738224,N,3536.7,434.5,85,67,76.0,0.0,0.0,0.0


In [47]:
df_copy.to_csv('../_data/train_with_osrm_and_weather_h.csv')

## * m_d

In [57]:
df_copy['pickup_datetime'] = df_copy['pickup_datetime'].apply(pd.to_datetime)

In [58]:
df_copy.insert(loc=4, column='m_d', value=np.vectorize(str)(df_copy['pickup_datetime'].dt.month*100
                                                             + df_copy['pickup_datetime'].dt.day))

In [59]:
df_copy.head()

,date,id,Time,vendor_id,m_d,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,total_distance,total_travel_time,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,2016-06-30,id2332349,2016-06-30 23:00:00,2,630,2016-06-30 23:59:00,5,-73.998291,40.722618,-73.971779,40.762470,N,5280.3,623.3,85,67,76.0,0.0,0.0,0.0
1,2016-06-30,id2013516,2016-06-30 23:00:00,1,630,2016-06-30 23:59:00,2,-73.955109,40.689564,-73.978203,40.685802,N,2573.5,219.3,85,67,76.0,0.0,0.0,0.0
2,2016-06-30,id3719493,2016-06-30 23:00:00,2,630,2016-06-30 23:59:00,1,-73.978416,40.791576,-73.973160,40.675968,N,15048.8,1300.0,85,67,76.0,0.0,0.0,0.0
3,2016-06-30,id0376262,2016-06-30 23:00:00,1,630,2016-06-30 23:59:00,2,-73.873093,40.774097,-73.926704,40.856739,N,16818.3,1029.1,85,67,76.0,0.0,0.0,0.0
4,2016-06-30,id2105228,2016-06-30 23:00:00,1,630,2016-06-30 23:58:00,1,-73.979416,40.755211,-74.003548,40.738224,N,3536.7,434.5,85,67,76.0,0.0,0.0,0.0


## * hr

In [60]:
df_copy.insert(loc=6, column='hr', value=df_copy['pickup_datetime'].dt.hour)

In [61]:
df_copy.head()

,date,id,Time,vendor_id,m_d,pickup_datetime,hr,passenger_count,pickup_longitude,pickup_latitude,...,dropoff_latitude,store_and_fwd_flag,total_distance,total_travel_time,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,2016-06-30,id2332349,2016-06-30 23:00:00,2,630,2016-06-30 23:59:00,23,5,-73.998291,40.722618,...,40.762470,N,5280.3,623.3,85,67,76.0,0.0,0.0,0.0
1,2016-06-30,id2013516,2016-06-30 23:00:00,1,630,2016-06-30 23:59:00,23,2,-73.955109,40.689564,...,40.685802,N,2573.5,219.3,85,67,76.0,0.0,0.0,0.0
2,2016-06-30,id3719493,2016-06-30 23:00:00,2,630,2016-06-30 23:59:00,23,1,-73.978416,40.791576,...,40.675968,N,15048.8,1300.0,85,67,76.0,0.0,0.0,0.0
3,2016-06-30,id0376262,2016-06-30 23:00:00,1,630,2016-06-30 23:59:00,23,2,-73.873093,40.774097,...,40.856739,N,16818.3,1029.1,85,67,76.0,0.0,0.0,0.0
4,2016-06-30,id2105228,2016-06-30 23:00:00,1,630,2016-06-30 23:58:00,23,1,-73.979416,40.755211,...,40.738224,N,3536.7,434.5,85,67,76.0,0.0,0.0,0.0


## * L2

In [41]:
def dist(plng, plat, dlng, dlat):
    return ((dlng-plng)**2 + (dlat-plat)**2) ** 0.5

In [42]:
df_copy.insert(loc=7, column='L2', value=np.vectorize(dist)(\
                df_copy['pickup_longitude'], df_copy['pickup_latitude'],\
                df_copy['dropoff_longitude'], df_copy['dropoff_latitude']))

df_copy.head()

,id,vendor_id,Time,pickup_datetime,m_d,dow,hr,L2,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,...,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,0.069424,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,0.057791,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,0.066698,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,23,0.018223,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,23,0.027198,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


## * holiday

In [62]:
holidays = {(2016, 1, 1) : 'New Year\'s',
            (2016, 1, 18) : 'MLK',
            (2016, 2, 15) : 'Washington',
            (2016, 5, 30) : 'Memorial' }

def is_holiday(timestamp):
    return (timestamp.year, timestamp.month, timestamp.day) in holidays

t = pd.Timestamp(2016, 5, 30)

print(is_holiday(t))

True


In [63]:
s_hol = df_copy['pickup_datetime'].apply(is_holiday)
s_hol.sum()

4057

In [64]:
df_copy.insert(loc=4, column='is_hol', value=s_hol)

In [65]:
df_copy.head()

,date,id,Time,vendor_id,is_hol,m_d,pickup_datetime,hr,passenger_count,pickup_longitude,...,dropoff_latitude,store_and_fwd_flag,total_distance,total_travel_time,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,2016-06-30,id2332349,2016-06-30 23:00:00,2,False,630,2016-06-30 23:59:00,23,5,-73.998291,...,40.762470,N,5280.3,623.3,85,67,76.0,0.0,0.0,0.0
1,2016-06-30,id2013516,2016-06-30 23:00:00,1,False,630,2016-06-30 23:59:00,23,2,-73.955109,...,40.685802,N,2573.5,219.3,85,67,76.0,0.0,0.0,0.0
2,2016-06-30,id3719493,2016-06-30 23:00:00,2,False,630,2016-06-30 23:59:00,23,1,-73.978416,...,40.675968,N,15048.8,1300.0,85,67,76.0,0.0,0.0,0.0
3,2016-06-30,id0376262,2016-06-30 23:00:00,1,False,630,2016-06-30 23:59:00,23,2,-73.873093,...,40.856739,N,16818.3,1029.1,85,67,76.0,0.0,0.0,0.0
4,2016-06-30,id2105228,2016-06-30 23:00:00,1,False,630,2016-06-30 23:58:00,23,1,-73.979416,...,40.738224,N,3536.7,434.5,85,67,76.0,0.0,0.0,0.0


## * disaster

??

In [69]:
df_copy.to_csv('../_data/train_with__all_data_no_disaster.csv', index=False)

---

In [ ]:
# p_lng is idx2
# ...
# d_lat is idx5

filtered_df = taxi_df

for i in range(2, 6):
    lower = np.percentile(taxi_df.iloc[:, i], 0.3)
    upper = np.percentile(taxi_df.iloc[:, i], 99.7)
    filtered_df = filtered_df[ (filtered_df.iloc[:, i] > lower) \
                         & (filtered_df.iloc[:, i] < upper) ]

filtered_df.shape
  

In [70]:
pd.options.display.max_columns = 60

In [2]:
df_copy = pd.read_csv('../_data/train_with__all_data_no_disaster.csv')

In [3]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,is_hol,m_d,dow,hr,L2,dropoff_datetime,...,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.069424,5/1/16 0:24,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.057791,5/1/16 0:22,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.066698,5/1/16 0:17,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,False,430,Saturday,23,0.018223,5/1/16 0:12,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,False,430,Saturday,23,0.027198,5/1/16 0:17,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


In [84]:
droppable0 = ['id', 'vendor_id', 'Time', 'pickup_datetime', 'dropoff_datetime',\
             'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',\
             'store_and_fwd_flag', 'starting_street', 'end_street',\
             'number_of_steps', 'street_for_each_step', 'distance_per_step', 'travel_time_per_step',\
             'step_maneuvers', 'step_direction', 'step_location_list',\
             'average temperature', 'Wind Dir', 'Events', 'Conditions']

droppable = ['id', 'vendor_id', 'Time', 'pickup_datetime', \
             'store_and_fwd_flag', 'starting_street', 'end_street',\
             'number_of_steps', 'street_for_each_step', 'distance_per_step', 'travel_time_per_step',\
             'step_maneuvers', 'step_direction', 'step_location_list',\
             'average temperature', 'Wind Dir', 'Events', 'Conditions']

In [86]:
for col in droppable:
    try:
        del df_copy[col]
    except:
        continue

In [87]:
df_copy.head()

,date,is_hol,m_d,hr,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_distance,total_travel_time,maximum temperature,minimum temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Speed,Gust Speed,Precip
0,2016-06-30,False,630,23,5,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,2016-06-30,False,630,23,2,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,2016-06-30,False,630,23,1,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,2016-06-30,False,630,23,2,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,2016-06-30,False,630,23,1,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


In [7]:
df_copy.to_csv('../_data/train__dropped__lnglat_saved__no_disaster.csv', index=False)

In [8]:
df_copy.columns

Index(['is_hol', 'm_d', 'dow', 'hr', 'L2', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'trip_duration', 'total_distance',
       'total_travel_time', 'maximum temperature', 'minimum temperature',
       'precipitation', 'snow fall', 'snow depth', 'Temp.', 'Humidity',
       'Dew Point', 'Wind Speed', 'Gust Speed', 'Precip'],
      dtype='object')

In [35]:
df_copy = pd.read_csv('../_data/train__dropped__lnglat_saved__no_disaster.csv')

In [94]:
short_cols = ['date', 'is_hol', 'm_d', 'hr', 'pas_cnt', 'p_lng', 'p_lat', 'd_lng', 'd_lat',
       'total_dist', 'total_time', 'day_max_temp',
       'day_min_temp', 'day_rainfall', 'day_snowfall', 'day_snowdepth',
       'hr_temp', 'hr_humid', 'hr_dewpt', 'hr_windspeed', 'hr_gustspeed', 'hr_precip']

In [95]:
df_copy.columns = short_cols

In [66]:
pd.options.display.max_columns = 60

In [96]:
df_copy.head()

,date,is_hol,m_d,hr,pas_cnt,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,2016-06-30,False,630,23,5,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,2016-06-30,False,630,23,2,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,2016-06-30,False,630,23,1,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,2016-06-30,False,630,23,2,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,2016-06-30,False,630,23,1,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


In [97]:
df_copy['m_d'] = df_copy['m_d'].apply(str)

In [98]:
df_copy.isnull().sum()

date                0
is_hol              0
m_d                 0
hr                  0
pas_cnt             0
p_lng               0
p_lat               0
d_lng               0
d_lat               0
total_dist          0
total_time          0
day_max_temp        0
day_min_temp        0
day_rainfall        0
day_snowfall        0
day_snowdepth       0
hr_temp          3354
hr_humid         3354
hr_dewpt         3354
hr_windspeed     3354
hr_gustspeed     3354
hr_precip        3354
dtype: int64

In [99]:
nans = df_copy.loc[df_copy['hr_temp'].isnull(), ['m_d', 'hr']].copy()
nans['m_d'] = nans['m_d'].apply(int)
print(nans.head())
nansarray = np.array(nans)
np.unique(nansarray, axis=0)

        m_d  hr
261602  515   7
261603  515   7
261604  515   7
261605  515   7
261606  515   7


array([[512,  16],
       [512,  17],
       [513,   3],
       [513,  20],
       [514,   0],
       [514,   1],
       [514,  12],
       [514,  13],
       [514,  14],
       [515,   0],
       [515,   1],
       [515,   7]], dtype=int64)

In [100]:
nan_pairs = np.unique(nansarray, axis=0)

nan_pairs

array([[512,  16],
       [512,  17],
       [513,   3],
       [513,  20],
       [514,   0],
       [514,   1],
       [514,  12],
       [514,  13],
       [514,  14],
       [515,   0],
       [515,   1],
       [515,   7]], dtype=int64)

In [75]:
df_copy['m_d']

0         630
1         630
2         630
3         630
4         630
5         630
6         630
7         630
8         630
9         630
10        630
11        630
12        630
13        630
14        630
15        630
16        630
17        630
18        630
19        630
20        630
21        630
22        630
23        630
24        630
25        630
26        630
27        630
28        630
29        630
         ... 
346767    501
346768    501
346769    501
346770    501
346771    501
346772    501
346773    501
346774    501
346775    501
346776    501
346777    501
346778    501
346779    501
346780    501
346781    501
346782    501
346783    501
346784    501
346785    501
346786    501
346787    501
346788    501
346789    501
346790    501
346791    501
346792    501
346793    501
346794    501
346795    501
346796    501
Name: m_d, Length: 346797, dtype: object

In [101]:
df_copy2 = df_copy.copy()

In [102]:
def get_prev(md, hr):
    m = int(md[0])
    d = int(md[1:])
    ts = pd.Timestamp(2016, m, d, hr, 0, 0)
    ts -= pd.Timedelta(hours=1)
    m_new = ts.month
    d_new = ts.day
    hr_new = ts.hour
    md_new = str(m_new)+'{:02}'.format(d_new)
    return(md_new, hr_new)

def get_next(m_d, hr):
    m = int(m_d[0])
    d = int(m_d[1:])
    ts = pd.Timestamp(2016, m, d, hr, 0, 0)
    ts += pd.Timedelta(hours=1)
    m_new = ts.month
    d_new = ts.day
    hr_new = ts.hour
    m_d_new = str(m_new)+'{:02}'.format(d_new)
    return(m_d_new, hr_new)

get_prev('105', 0)
get_next('630', 23)

('701', 0)

In [103]:
df_copy2['m_d'].head()


0    630
1    630
2    630
3    630
4    630
Name: m_d, dtype: object

In [106]:
df_copy2['hr_temp'][(df_copy2['m_d'] == md_prev) & (df_copy2['hr'] == hr_prev)].isnull().sum()

0

In [105]:
fill_cols = ['hr_temp', 'hr_humid', 'hr_dewpt', 'hr_windspeed', 'hr_gustspeed', 'hr_precip']

for md, hr in nan_pairs:
    print(md, hr)
    md = str(md)
    md_prev, hr_prev = md, hr
    md_next, hr_next = md, hr
    while df_copy2['hr_temp'][(df_copy2['m_d'] == md_prev) & (df_copy2['hr'] == hr_prev)].isnull().sum() > 0:
        md_prev, hr_prev = get_prev(md_prev, hr_prev)
    while df_copy2['hr_temp'][(df_copy2['m_d'] == md_next) & (df_copy2['hr'] == hr_next)].isnull().sum() > 0:
        md_next, hr_next = get_next(md_next, hr_next)

    for col in fill_cols:
        print(col)
        col_prev = df_copy2[col][(df_copy2['m_d'] == md_prev) & (df_copy2['hr'] == hr_prev)].iloc[0]
        col_next = df_copy2[col][(df_copy2['m_d'] == md_next) & (df_copy2['hr'] == hr_next)].iloc[0]
        col_new = np.average([col_prev, col_next])
        print(col_new)
        df_copy2.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), col] = col_new
    print()


512 16
hr_temp
24.7
hr_humid
0.24
hr_dewpt
2.8
hr_windspeed
2.8
hr_gustspeed
0.0
hr_precip
0.0

512 17
hr_temp
24.0
hr_humid
0.26
hr_dewpt
3.35
hr_windspeed
4.2
hr_gustspeed
0.0
hr_precip
0.0

513 3
hr_temp
14.4
hr_humid
0.72
hr_dewpt
9.4
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

513 20
hr_temp
16.95
hr_humid
0.97
hr_dewpt
16.45
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

514 0
hr_temp
16.25
hr_humid
0.985
hr_dewpt
16.05
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

514 1
hr_temp
15.875
hr_humid
0.9925
hr_dewpt
15.775
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

514 12
hr_temp
21.5
hr_humid
0.475
hr_dewpt
10.0
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

514 13
hr_temp
21.75
hr_humid
0.4675
hr_dewpt
10.0
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

514 14
hr_temp
21.875
hr_humid
0.46375
hr_dewpt
10.0
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

515 0
hr_temp
15.5
hr_humid
0.7
hr_dewpt
10.0
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

515 1
hr_temp
15.

In [107]:
df_copy2.head()

,date,is_hol,m_d,hr,pas_cnt,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,2016-06-30,False,630,23,5,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,2016-06-30,False,630,23,2,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,2016-06-30,False,630,23,1,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,2016-06-30,False,630,23,2,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,2016-06-30,False,630,23,1,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


In [109]:
df_copy2.to_csv('../_data/test_backup_raw_osrm_dailyw_hourlyw.csv', index=False)

In [108]:
df_copy3 = df_copy2.copy()
df_copy3.isnull().sum()

date             0
is_hol           0
m_d              0
hr               0
pas_cnt          0
p_lng            0
p_lat            0
d_lng            0
d_lat            0
total_dist       0
total_time       0
day_max_temp     0
day_min_temp     0
day_rainfall     0
day_snowfall     0
day_snowdepth    0
hr_temp          0
hr_humid         0
hr_dewpt         0
hr_windspeed     0
hr_gustspeed     0
hr_precip        0
dtype: int64

In [236]:
md = '419'
hr = 11
df_copy2.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), 'hr_temp'].head()

67827    19.45
67828    19.45
67829    19.45
67830    19.45
67831    19.45
Name: hr_temp, dtype: float64

In [234]:
df_copy3.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), 'hr_temp'] = 3
df_copy3.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), 'hr_temp'].head()

67827    3.0
67828    3.0
67829    3.0
67830    3.0
67831    3.0
Name: hr_temp, dtype: float64

In [162]:
ssss[:]

67827   NaN
67828   NaN
67829   NaN
67830   NaN
67831   NaN
67832   NaN
67833   NaN
67834   NaN
67835   NaN
67836   NaN
67837   NaN
67838   NaN
67839   NaN
67840   NaN
67841   NaN
67842   NaN
67843   NaN
67844   NaN
67845   NaN
67846   NaN
67847   NaN
67848   NaN
67849   NaN
67850   NaN
67851   NaN
67852   NaN
67853   NaN
67854   NaN
67855   NaN
67856   NaN
         ..
68066   NaN
68067   NaN
68068   NaN
68069   NaN
68070   NaN
68071   NaN
68072   NaN
68073   NaN
68074   NaN
68075   NaN
68076   NaN
68077   NaN
68078   NaN
68079   NaN
68080   NaN
68081   NaN
68082   NaN
68083   NaN
68084   NaN
68085   NaN
68086   NaN
68087   NaN
68088   NaN
68089   NaN
68090   NaN
68091   NaN
68092   NaN
68093   NaN
68094   NaN
68095   NaN
Name: hr_temp, Length: 269, dtype: float64

In [ ]:
df_copy.to_csv('../_data/train__dropped__no_disaster.csv', index=False)

In [38]:
def make_hourly(timestamp):
    y = timestamp.year
    m = timestamp.month
    d = timestamp.day
    h = timestamp.hour
    return pd.Timestamp(y, m, d, h, 0, 0)

In [45]:
df_copy.insert(2, 'Time', df_copy['pickup_datetime'].apply(make_hourly))

In [44]:
del df_copy['pickup_hour']

In [46]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,...,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,...,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,...,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,...,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0


In [47]:
df_copy.to_csv('../_data/train_with_osrm_and_weather_h.csv')

In [42]:
df_hourly.head()

,Time,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,2015-12-31 02:00:00,7.8,0.89,6.1,NNE,5.6,0.0,0.8,None,Overcast
1,2015-12-31 03:00:00,7.2,0.90,5.6,Variable,7.4,0.0,0.3,None,Overcast
2,2015-12-31 04:00:00,7.2,0.90,5.6,Calm,0.0,0.0,0.0,None,Overcast
3,2015-12-31 05:00:00,7.2,0.86,5.0,NW,7.4,0.0,0.0,None,Overcast
4,2015-12-31 06:00:00,7.2,0.90,5.6,West,5.6,0.0,0.0,None,Overcast


In [48]:
df_merged = df_copy.merge(df_hourly, how='left', on='Time')

In [64]:
df_hourly.iloc[2901]

Time          2016-04-30 23:00:00
Temp.                        10.6
Humidity                     0.56
Dew Point                     2.2
Wind Dir                      SSE
Wind Speed                     13
Gust Speed                      0
Precip                          0
Events                       None
Conditions                  Clear
Name: 2901, dtype: object

In [49]:
df_merged.head()

,id,vendor_id,Time,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,...,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


### df + osrm + weather + hourly weather 저장

In [65]:
df_merged.to_csv('../_data/train_with_osrm_and_weather_hourly.csv')

In [42]:
df_hourly.head()

,Time,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,2015-12-31 02:00:00,7.8,0.89,6.1,NNE,5.6,0.0,0.8,None,Overcast
1,2015-12-31 03:00:00,7.2,0.90,5.6,Variable,7.4,0.0,0.3,None,Overcast
2,2015-12-31 04:00:00,7.2,0.90,5.6,Calm,0.0,0.0,0.0,None,Overcast
3,2015-12-31 05:00:00,7.2,0.86,5.0,NW,7.4,0.0,0.0,None,Overcast
4,2015-12-31 06:00:00,7.2,0.90,5.6,West,5.6,0.0,0.0,None,Overcast


In [48]:
df_merged = df_copy.merge(df_hourly, how='left', on='Time')

In [64]:
df_hourly.iloc[2901]

Time          2016-04-30 23:00:00
Temp.                        10.6
Humidity                     0.56
Dew Point                     2.2
Wind Dir                      SSE
Wind Speed                     13
Gust Speed                      0
Precip                          0
Events                       None
Conditions                  Clear
Name: 2901, dtype: object

In [49]:
df_merged.head()

,id,vendor_id,Time,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,...,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


### df + osrm + weather + hourly weather 저장

In [65]:
df_merged.to_csv('../_data/train_with_osrm_and_weather_hourly.csv', index=False)